In [2]:
import os
import time
import pandas as pd
import numpy as np
from random import sample

import torch
import torch.nn as nn
import torch.nn.functional as F

from Bio import SeqIO
from multiprocessing import Process

import matplotlib.pyplot as plt

In [3]:
conv_d = {'A': [0],
          'T': [1],
          'C': [2],
          'G': [3],
          'Y': [1, 2], # T, C
          'R': [0, 3], # A, G
          'S': [2, 3], # C, G
          'W': [0, 1], # A, T
          'K': [1, 3], # G, T
          'M': [0, 2], # A, C
          'B': [1, 2, 3], # T, C, G
          'D': [0, 1, 3], # A, T, G
          'H': [0, 1, 2], # A, T, C
          'V': [0, 2, 3], # A, C, G
          'N': [0, 1, 2, 3]
#           '-': None
         }

# input core genome matirix making

In [4]:
core_genome_fasta = SeqIO.to_dict(SeqIO.parse('the_core_genome_1127_full.fasta',
                                              'fasta'
                                             )
                             )

In [5]:
np_path = 'np/'

def DNA_to_np(core_genomes, in_lst, convert_dict):
    check_lst = list(convert_dict.keys())
    
    for f in in_lst:
        s_seq = str(core_genomes[f].seq) 
        
        b_seq = np.zeros([4, len(s_seq)], 
                     dtype = 'int'
                    )
        # vlauing
        for x in range(len(s_seq)):
            if s_seq[x] not in check_lst:
                continue
            else:
                length = len(convert_dict[s_seq[x]])
                
                for y in convert_dict[s_seq[x]]:
                    b_seq[convert_dict[s_seq[x]], x] = 1/length

        with open(np_path
                  + f 
                  + '.npy',
                  'wb') as f:
            np.save(f, b_seq)

In [6]:
start_time_external = time.time()
print(time.asctime())

if __name__ == '__main__':
    procc_num = 16
    
    mission_lst = np.array_split(list(core_genome_fasta.keys()), 
                                 procc_num
                                )
    
    jobs = []
    
    for x in range(0, procc_num):
        p = Process(target = DNA_to_np,
                    args = (core_genome_fasta, 
                            mission_lst[x], 
                            conv_d
                           )
                   )
        p.start()
        jobs.append(p)
        
    
    for z in jobs:
        z.join()
        
end_time_external = time.time()
print("final time usage " + str(end_time_external - start_time_external))
print(time.asctime())

Sun Jan 29 13:16:10 2023
final time usage 198.07088661193848
Sun Jan 29 13:19:28 2023


# input core genome matirix making ---- genome before alignment

In [12]:
core_genome_fasta = SeqIO.to_dict(SeqIO.parse('/data/docker_qiime2_share_G/Hejunwei/core_genome_study/'
                                              + 'CNN/the_core_genome_1127_no_align.fasta',
                                              'fasta'
                                             )
                             )

In [13]:
largest_finder = []

for x in core_genome_fasta.keys():
    largest_finder.append(len(core_genome_fasta[x].seq))

In [14]:
max(largest_finder)

721826

In [18]:
np_path = '/data/docker_qiime2_share_G/Hejunwei/core_genome_study/CNN/np/'

def DNA_to_np(core_genomes, in_lst, convert_dict):
    check_lst = list(convert_dict.keys())
    
    for f in in_lst:
        s_seq = str(core_genomes[f].seq) 
        
        b_seq = np.zeros([4, len(s_seq)], 
                         dtype = 'int'
                        )
        
        pad_num = 721826 - len(s_seq)
        
        pad_front = pad_num // 2

        pad_end = pad_num - pad_front
        
        # vlauing
        for x in range(len(s_seq)):
            if s_seq[x] not in check_lst:
                continue
            else:
                length = len(convert_dict[s_seq[x]])
                
                for y in convert_dict[s_seq[x]]:
                    b_seq[convert_dict[s_seq[x]], x] = 1/length
        
        b_seq_tosave = np.c_[np.zeros([4, pad_front], dtype = 'int'), 
                             b_seq,
                             np.zeros([4, pad_end], dtype = 'int')
                            ]

        with open(np_path
                  + f 
                  + '.npy',
                  'wb') as f:
            np.save(f, b_seq_tosave)

In [19]:
start_time_external = time.time()
print(time.asctime())

if __name__ == '__main__':
    procc_num = 36
    
    mission_lst = np.array_split(list(core_genome_fasta.keys()), 
                                 procc_num
                                )
    
    jobs = []
    
    for x in range(0, procc_num):
        p = Process(target = DNA_to_np,
                    args = (core_genome_fasta, 
                            mission_lst[x], 
                            conv_d
                           )
                   )
        p.start()
        jobs.append(p)
        
    
    for z in jobs:
        z.join()
        
end_time_external = time.time()
print("final time usage " + str(end_time_external - start_time_external))
print(time.asctime())

Mon Nov 14 11:45:01 2022
final time usage 90.3128764629364
Mon Nov 14 11:46:31 2022


# KO

In [36]:
core_genome_fasta = SeqIO.to_dict(SeqIO.parse('/data/docker_qiime2_share_G/Hejunwei/core_genome_study/'
                                              + 'CNN/the_core_genome_1127_KO.fasta',
                                              'fasta'
                                             )
                             )

In [37]:
np_path = '/data/docker_qiime2_share_G/Hejunwei/core_genome_study/CNN/np_KO/'

def DNA_to_np(core_genomes, in_lst, convert_dict):
    check_lst = list(convert_dict.keys())
    
    for f in in_lst:
        s_seq = str(core_genomes[f].seq) 
        
        b_seq = np.zeros([4, len(s_seq)], 
                         dtype = 'int'
                        )
        
        # vlauing
        for x in range(len(s_seq)):
            if s_seq[x] not in check_lst:
                continue
            else:
                length = len(convert_dict[s_seq[x]])
                
                for y in convert_dict[s_seq[x]]:
                    b_seq[convert_dict[s_seq[x]], x] = 1/length

        with open(np_path
                  + f 
                  + '.npy',
                  'wb') as f:
            np.save(f, b_seq)

In [38]:
start_time_external = time.time()
print(time.asctime())

if __name__ == '__main__':
    procc_num = 36
    
    mission_lst = np.array_split(list(core_genome_fasta.keys()), 
                                 procc_num
                                )
    
    jobs = []
    
    for x in range(0, procc_num):
        p = Process(target = DNA_to_np,
                    args = (core_genome_fasta, 
                            mission_lst[x], 
                            conv_d
                           )
                   )
        p.start()
        jobs.append(p)
        
    
    for z in jobs:
        z.join()
        
end_time_external = time.time()
print("final time usage " + str(end_time_external - start_time_external))
print(time.asctime())

Thu Nov 17 04:40:08 2022
final time usage 41.139267444610596
Thu Nov 17 04:40:49 2022
